In [140]:
# Importing necessary packages
import torch
import math
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms, models
import torch.nn.functional as F
import time
import os
import torch.backends.cudnn as cudnn
import torchvision.models as models
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import torchvision
import pandas as pd
import matplotlib.pyplot as plt

In [141]:
# Initializing parameters
os.environ["CUDA_VISIBLE_DEVICES"] = '0'                # GPU Number 
start_time = time.time()
batch_size = 40
learning_rate = 0.001
num_of_epochs = 15
root_dir = 'C/Users/MYDocuments/Deep Learning Project/VGG_shahg1'
default_directory = 'C/Users/MYDocuments/Deep Learning Project/VGG_shahg1'
val_losses = []
train_losses = []
#

In [142]:
# Defining Data Augmentation
transform_train = transforms.Compose([
    transforms.Resize(size=(224, 224)),               # Random Position Crop
    transforms.RandomHorizontalFlip(),                  # right and left flip
    transforms.ToTensor(),                              # change [0,255] Int value to [0,1] Float value
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010) )  # RGB Normalize Standard Deviation
])

transform_test = transforms.Compose([
    transforms.Resize(size=(224, 224)),                               
    transforms.ToTensor(),                              # change [0,255] Int value to [0,1] Float value
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010) )  # RGB Normalize Standard Deviation
])

#

In [143]:
 # Automatic downloading datasets and data augmentation
train_dataset = datasets.CIFAR10(root=root_dir,
                                 train=True,
                                 transform=transform_train,
                                 download=True)

test_dataset = datasets.CIFAR10(root=root_dir,
                                train=False,
                                transform=transform_test)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,            # at Training Procedure, Data Shuffle = True
                                           num_workers=4)           # CPU loader number

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,            # at Test Procedure, Data Shuffle = False
                                          num_workers=4)            # CPU loader number




Files already downloaded and verified


In [144]:
# Setting up the processing device and initializing the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def initialize_model( num_classes, use_pretrained=True):

    model_ft = models.vgg19_bn(pretrained=use_pretrained)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
    input_size = 224
    return model_ft, input_size

# Initialize the model for this run
model, input_size = initialize_model(10, use_pretrained=True)
'''
#loading Pretrainined VGG
model= models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, 10)

'''


'\n#loading Pretrainined VGG\nmodel= models.vgg16(pretrained=True)\nmodel.classifier[6] = nn.Linear(4096, 10)\n\n'

In [145]:
# Defining optimizer
model = model.to(device)
#optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
optimizer = optim.Adagrad(model.parameters(), lr=0.01, lr_decay=0, weight_decay=0, initial_accumulator_value=0, eps=1e-10) #Changed the optimizer to Adagrad 


criterion = nn.CrossEntropyLoss()

if torch.cuda.device_count() > 0:
    print("USE", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model).cuda()
    cudnn.benchmark = True
else:
    print("USE ONLY CPU!")



USE ONLY CPU!


In [146]:
# Defining the train function
def train(epoch):
    model.train()
    train_loss = 0 
    total = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = Variable(data.cuda()), Variable(target.cuda())
        else:
            data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        writer.add_scalar('Loss/train', train_loss, epoch)
        
        total += target.size(0)
        correct += predicted.eq(target.data).cpu().sum()
        if batch_idx % 10 == 0:
            print('Epoch: {} | Batch_idx: {} |  Loss: ({:.4f}) | Acc: ({:.2f}%) ({}/{})'
                  .format(epoch, batch_idx, train_loss / (batch_idx + 1), 100. * correct / total, correct, total))
        
        TRAIN_LOSS.append(train_loss)
        TRAIN_ACC.append((100. * correct / total))

            
    dict = {'train loss' : TRAIN_LOSS, 'train accuracy' : (100. * correct / total)}
    df = pd.DataFrame(dict)
    df.to_csv('train_loss_and_acc_vgg.csv')
        



In [147]:
# Defining the test function
def test():
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        if torch.cuda.is_available():
            data, target = Variable(data.cuda()), Variable(target.cuda())
        else:
            data, target = Variable(data), Variable(target)

        outputs = model(data)
        loss = criterion(outputs, target)

        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        writer.add_scalar('Loss/val', test_loss, epoch)
        
        total += target.size(0)
        correct += predicted.eq(target.data).cpu().sum()
        
        TEST_LOSS.append(test_loss)
        TEST_ACC.append((100. * correct / total))
        
    print('# TEST : Loss: ({:.4f}) | Acc: ({:.2f}%) ({}/{})'
          .format(test_loss / (batch_idx + 1), 100. * correct / total, correct, total))


            
    dict = {'test loss' : TEST_LOSS, 'train accuracy' : (100. * correct / total)}
    df = pd.DataFrame(dict)
    df.to_csv('test_loss_and_acc_vgg.csv')


def save_checkpoint(directory, state, filename='latest.tar.gz'):

    if not os.path.exists(directory):
        os.makedirs(directory)

    model_filename = os.path.join(directory, filename)
    torch.save(state, model_filename)
    print("=> saving checkpoint")

def load_checkpoint(directory, filename='latest.tar.gz'):

    model_filename = os.path.join(directory, filename)
    if os.path.exists(model_filename):
        print("=> loading checkpoint")
        state = torch.load(model_filename)
        return state
    else:
        return None

In [ ]:
# Main function for training the model

TRAIN_LOSS = []
TEST_LOSS = []
TRAIN_ACC = []
TEST_ACC = []

start_epoch = 0

checkpoint = load_checkpoint(default_directory)
if not checkpoint:
    pass
else:
    start_epoch = checkpoint['epoch'] + 1
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

writer = SummaryWriter(root_dir)    

for epoch in range(start_epoch, num_of_epochs):

    if epoch < 20:
        lr = learning_rate
    elif epoch < 40:
        lr = learning_rate * 0.1
    else:
        lr = learning_rate * 0.01
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    train(epoch)
    save_checkpoint(default_directory, {
        'epoch': epoch,
        'model': model,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    })
    test()  

writer.close()
    
now = time.gmtime(time.time() - start_time)
print('{} hours {} mins {} secs for training'.format(now.tm_hour, now.tm_min, now.tm_sec))

=> loading checkpoint
Epoch: 2 | Batch_idx: 0 |  Loss: (0.0587) | Acc: (100.00%) (40/40)
Epoch: 2 | Batch_idx: 10 |  Loss: (0.0555) | Acc: (98.41%) (433/440)
Epoch: 2 | Batch_idx: 20 |  Loss: (0.0591) | Acc: (98.33%) (826/840)
Epoch: 2 | Batch_idx: 30 |  Loss: (0.0519) | Acc: (98.55%) (1222/1240)
Epoch: 2 | Batch_idx: 40 |  Loss: (0.0585) | Acc: (98.29%) (1612/1640)
Epoch: 2 | Batch_idx: 50 |  Loss: (0.0552) | Acc: (98.38%) (2007/2040)
Epoch: 2 | Batch_idx: 60 |  Loss: (0.0538) | Acc: (98.36%) (2400/2440)
Epoch: 2 | Batch_idx: 70 |  Loss: (0.0531) | Acc: (98.45%) (2796/2840)
Epoch: 2 | Batch_idx: 80 |  Loss: (0.0557) | Acc: (98.36%) (3187/3240)
Epoch: 2 | Batch_idx: 90 |  Loss: (0.0568) | Acc: (98.30%) (3578/3640)
Epoch: 2 | Batch_idx: 100 |  Loss: (0.0581) | Acc: (98.27%) (3970/4040)
Epoch: 2 | Batch_idx: 110 |  Loss: (0.0599) | Acc: (98.20%) (4360/4440)
Epoch: 2 | Batch_idx: 120 |  Loss: (0.0582) | Acc: (98.24%) (4755/4840)
Epoch: 2 | Batch_idx: 130 |  Loss: (0.0573) | Acc: (98.26%) 

Epoch: 2 | Batch_idx: 1120 |  Loss: (0.0649) | Acc: (97.87%) (43883/44840)
Epoch: 2 | Batch_idx: 1130 |  Loss: (0.0649) | Acc: (97.86%) (44274/45240)
Epoch: 2 | Batch_idx: 1140 |  Loss: (0.0652) | Acc: (97.86%) (44662/45640)
Epoch: 2 | Batch_idx: 1150 |  Loss: (0.0652) | Acc: (97.86%) (45057/46040)
Epoch: 2 | Batch_idx: 1160 |  Loss: (0.0651) | Acc: (97.86%) (45448/46440)
Epoch: 2 | Batch_idx: 1170 |  Loss: (0.0649) | Acc: (97.87%) (45843/46840)
Epoch: 2 | Batch_idx: 1180 |  Loss: (0.0650) | Acc: (97.87%) (46234/47240)
Epoch: 2 | Batch_idx: 1190 |  Loss: (0.0649) | Acc: (97.88%) (46629/47640)
Epoch: 2 | Batch_idx: 1200 |  Loss: (0.0646) | Acc: (97.88%) (47023/48040)
Epoch: 2 | Batch_idx: 1210 |  Loss: (0.0647) | Acc: (97.88%) (47415/48440)
Epoch: 2 | Batch_idx: 1220 |  Loss: (0.0648) | Acc: (97.88%) (47804/48840)
Epoch: 2 | Batch_idx: 1230 |  Loss: (0.0648) | Acc: (97.88%) (48198/49240)
Epoch: 2 | Batch_idx: 1240 |  Loss: (0.0648) | Acc: (97.89%) (48591/49640)
=> saving checkpoint
# TE

Epoch: 3 | Batch_idx: 980 |  Loss: (0.0390) | Acc: (98.77%) (38756/39240)
Epoch: 3 | Batch_idx: 990 |  Loss: (0.0390) | Acc: (98.77%) (39151/39640)
Epoch: 3 | Batch_idx: 1000 |  Loss: (0.0389) | Acc: (98.77%) (39548/40040)
Epoch: 3 | Batch_idx: 1010 |  Loss: (0.0389) | Acc: (98.77%) (39943/40440)
Epoch: 3 | Batch_idx: 1020 |  Loss: (0.0388) | Acc: (98.77%) (40339/40840)
Epoch: 3 | Batch_idx: 1030 |  Loss: (0.0392) | Acc: (98.77%) (40731/41240)
Epoch: 3 | Batch_idx: 1040 |  Loss: (0.0391) | Acc: (98.77%) (41127/41640)
Epoch: 3 | Batch_idx: 1050 |  Loss: (0.0391) | Acc: (98.77%) (41525/42040)
Epoch: 3 | Batch_idx: 1060 |  Loss: (0.0389) | Acc: (98.78%) (41924/42440)
Epoch: 3 | Batch_idx: 1070 |  Loss: (0.0387) | Acc: (98.79%) (42321/42840)
Epoch: 3 | Batch_idx: 1080 |  Loss: (0.0386) | Acc: (98.78%) (42714/43240)
Epoch: 3 | Batch_idx: 1090 |  Loss: (0.0387) | Acc: (98.78%) (43107/43640)
Epoch: 3 | Batch_idx: 1100 |  Loss: (0.0389) | Acc: (98.77%) (43498/44040)
Epoch: 3 | Batch_idx: 1110 

Epoch: 4 | Batch_idx: 840 |  Loss: (0.0257) | Acc: (99.23%) (33381/33640)
Epoch: 4 | Batch_idx: 850 |  Loss: (0.0257) | Acc: (99.23%) (33777/34040)
Epoch: 4 | Batch_idx: 860 |  Loss: (0.0257) | Acc: (99.22%) (34173/34440)
Epoch: 4 | Batch_idx: 870 |  Loss: (0.0259) | Acc: (99.22%) (34568/34840)
Epoch: 4 | Batch_idx: 880 |  Loss: (0.0258) | Acc: (99.22%) (34965/35240)
Epoch: 4 | Batch_idx: 890 |  Loss: (0.0260) | Acc: (99.22%) (35361/35640)
Epoch: 4 | Batch_idx: 900 |  Loss: (0.0261) | Acc: (99.21%) (35754/36040)
Epoch: 4 | Batch_idx: 910 |  Loss: (0.0265) | Acc: (99.20%) (36148/36440)
Epoch: 4 | Batch_idx: 920 |  Loss: (0.0265) | Acc: (99.20%) (36544/36840)
Epoch: 4 | Batch_idx: 930 |  Loss: (0.0265) | Acc: (99.20%) (36941/37240)
Epoch: 4 | Batch_idx: 940 |  Loss: (0.0264) | Acc: (99.20%) (37340/37640)
Epoch: 4 | Batch_idx: 950 |  Loss: (0.0265) | Acc: (99.20%) (37736/38040)
Epoch: 4 | Batch_idx: 960 |  Loss: (0.0266) | Acc: (99.20%) (38131/38440)
Epoch: 4 | Batch_idx: 970 |  Loss: (0.

Epoch: 5 | Batch_idx: 700 |  Loss: (0.0157) | Acc: (99.55%) (27913/28040)
Epoch: 5 | Batch_idx: 710 |  Loss: (0.0158) | Acc: (99.54%) (28309/28440)
Epoch: 5 | Batch_idx: 720 |  Loss: (0.0158) | Acc: (99.54%) (28707/28840)
Epoch: 5 | Batch_idx: 730 |  Loss: (0.0158) | Acc: (99.54%) (29105/29240)
Epoch: 5 | Batch_idx: 740 |  Loss: (0.0158) | Acc: (99.54%) (29503/29640)
Epoch: 5 | Batch_idx: 750 |  Loss: (0.0157) | Acc: (99.54%) (29901/30040)
Epoch: 5 | Batch_idx: 760 |  Loss: (0.0156) | Acc: (99.54%) (30300/30440)
Epoch: 5 | Batch_idx: 770 |  Loss: (0.0157) | Acc: (99.53%) (30696/30840)
Epoch: 5 | Batch_idx: 780 |  Loss: (0.0157) | Acc: (99.53%) (31092/31240)
Epoch: 5 | Batch_idx: 790 |  Loss: (0.0159) | Acc: (99.51%) (31486/31640)
Epoch: 5 | Batch_idx: 800 |  Loss: (0.0161) | Acc: (99.50%) (31881/32040)
Epoch: 5 | Batch_idx: 810 |  Loss: (0.0161) | Acc: (99.50%) (32279/32440)
Epoch: 5 | Batch_idx: 820 |  Loss: (0.0160) | Acc: (99.50%) (32677/32840)
Epoch: 5 | Batch_idx: 830 |  Loss: (0.

Epoch: 6 | Batch_idx: 560 |  Loss: (0.0119) | Acc: (99.59%) (22347/22440)
Epoch: 6 | Batch_idx: 570 |  Loss: (0.0120) | Acc: (99.59%) (22746/22840)
Epoch: 6 | Batch_idx: 580 |  Loss: (0.0119) | Acc: (99.59%) (23145/23240)
Epoch: 6 | Batch_idx: 590 |  Loss: (0.0118) | Acc: (99.59%) (23544/23640)
Epoch: 6 | Batch_idx: 600 |  Loss: (0.0118) | Acc: (99.60%) (23943/24040)
Epoch: 6 | Batch_idx: 610 |  Loss: (0.0117) | Acc: (99.60%) (24343/24440)
Epoch: 6 | Batch_idx: 620 |  Loss: (0.0117) | Acc: (99.61%) (24743/24840)
Epoch: 6 | Batch_idx: 630 |  Loss: (0.0116) | Acc: (99.61%) (25142/25240)
Epoch: 6 | Batch_idx: 640 |  Loss: (0.0119) | Acc: (99.61%) (25539/25640)
Epoch: 6 | Batch_idx: 650 |  Loss: (0.0119) | Acc: (99.60%) (25937/26040)
Epoch: 6 | Batch_idx: 660 |  Loss: (0.0118) | Acc: (99.61%) (26337/26440)
Epoch: 6 | Batch_idx: 670 |  Loss: (0.0118) | Acc: (99.61%) (26736/26840)
Epoch: 6 | Batch_idx: 680 |  Loss: (0.0118) | Acc: (99.61%) (27135/27240)
Epoch: 6 | Batch_idx: 690 |  Loss: (0.

Epoch: 7 | Batch_idx: 420 |  Loss: (0.0096) | Acc: (99.72%) (16793/16840)
Epoch: 7 | Batch_idx: 430 |  Loss: (0.0094) | Acc: (99.73%) (17193/17240)
Epoch: 7 | Batch_idx: 440 |  Loss: (0.0096) | Acc: (99.73%) (17592/17640)
Epoch: 7 | Batch_idx: 450 |  Loss: (0.0096) | Acc: (99.73%) (17991/18040)
Epoch: 7 | Batch_idx: 460 |  Loss: (0.0099) | Acc: (99.72%) (18388/18440)
Epoch: 7 | Batch_idx: 470 |  Loss: (0.0099) | Acc: (99.72%) (18787/18840)
Epoch: 7 | Batch_idx: 480 |  Loss: (0.0099) | Acc: (99.71%) (19185/19240)
Epoch: 7 | Batch_idx: 490 |  Loss: (0.0099) | Acc: (99.71%) (19583/19640)
Epoch: 7 | Batch_idx: 500 |  Loss: (0.0098) | Acc: (99.72%) (19983/20040)
Epoch: 7 | Batch_idx: 510 |  Loss: (0.0097) | Acc: (99.72%) (20382/20440)
Epoch: 7 | Batch_idx: 520 |  Loss: (0.0098) | Acc: (99.71%) (20779/20840)
Epoch: 7 | Batch_idx: 530 |  Loss: (0.0097) | Acc: (99.71%) (21179/21240)
Epoch: 7 | Batch_idx: 540 |  Loss: (0.0097) | Acc: (99.71%) (21578/21640)
Epoch: 7 | Batch_idx: 550 |  Loss: (0.